
<img src="https://forms.trendminer.com/hs-fs/hubfs/Logo.png?width=1060&name=Logo.png"  width="500" height="500">

<span style="font-family:Times New Roman; font-size:30px;">Step 1: Import some python packages for data science</span>

<span style="font-family:Times New Roman; font-size:15px;">TrendMiner Initialization script</span>

In [1]:
# Suggested data science package imports
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt

# TrendMiner package import
import trendminer
from trendminer.trendminer_client import TrendMinerClient

import os

token = os.environ["KERNEL_USER_TOKEN"]
serverUrl = os.environ["KERNEL_SERVER_URL"]

# Create TrendMiner API object
client = TrendMinerClient(token, serverUrl)

<span style="font-family:Times New Roman; font-size:30px;">Step 2: Import Timeseries Data from TrendHub View</span>

<span style="font-family:Times New Roman; font-size:15px;">1 min interpolated data in a dataframe</span>

In [2]:
# Loading TrendHub view: R101 View [MLHub]
from trendminer.views.views import Views

views = Views(client)
df = views.load_view('0bc05bde-6237-475a-a560-67836631ecc7')
df_train = pd.concat(df)

In [3]:
# Loading TrendHub view: R101 View [MLHub]
from trendminer.views.views import Views

views = Views(client)
df = views.load_view('0bc05bde-6237-475a-a560-67836631ecc7')

In [4]:
df_train

<span style="font-family:Times New Roman; font-size:30px;">Step 3: Split Data into Training and Testing Dataset</span>

In [5]:
from sklearn.model_selection import train_test_split
features = ["R101_FI1", "R101_FI2"]
X = df_train[features]
y = df_train["R101_PI"]

#Splitting the data using the train_test_split function.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True)

<span style="font-family:Times New Roman; font-size:30px;">Step 4: Train Model</span>

<span style="font-family:Times New Roman; font-size:15px;">Using a Gradient Booosting Regressor Model from **ScikitLearn**</span>

In [6]:
# Both Random Forest and Gradient Boosting are ensemble learning methods.
# Both are based on decision trees.

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

#Create model into as a pipeline
# This Pipeline function calls for a list of estimators and other operators in the form of tuples...
estimators = [('model', GradientBoostingRegressor())]
pipeline_obj = Pipeline(estimators)

#Training the model
myModel = pipeline_obj.fit(X_train, y_train)

In [9]:
PredictedY = myModel.predict([20.1, 2.3, 89, 90, 12, 33])

<span style="font-family:Times New Roman; font-size:30px;">Step 5: Evaluate Prediction Effectiveness</span>

In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
import statsmodels.api as sm


y_pred = myModel.predict(X_test)

print("MSE = ", "{:.3f}".format(mean_squared_error(y_test, y_pred)))
print("Max Error = ", "{:.3f}".format(max_error(y_test, y_pred)))
print("Mean Abs Error = ", "{:.3f}".format(mean_absolute_error(y_test, y_pred)))


print("\nCoefficient of Determination")
print("R2 =", "{:.3f}".format(r2_score(y_test, y_pred)))


<span style="font-family:Times New Roman; font-size:30px;">Step 6: Convert ScikitLearn Model Pipeline to PMML</span>

<span style="font-family:Times New Roman; font-size:15px;">**PMML** format accepted by TrendMiner</span>

In [8]:
# PMML

from nyoka import skl_to_pmml
import warnings
warnings.filterwarnings("ignore")

#Converting model to pmml
model_name = "R101_PI_GradiantBoosting"

skl_to_pmml(pipeline=myModel,
            col_names=features,
            target_name="PredictedPressure",
            pmml_f_name="hello.pmml",
            model_name=model_name,
            description="test_description")

<span style="font-family:Times New Roman; font-size:30px;">Step 7: Model Deployment</span>

In [9]:
from trendminer.ml.models import ZementisModels

#Deploying model with Zementis
with open("hello.pmml","r") as f:
    string = f.read()

# But first, lets delete the model from pervious demo
zementis = ZementisModels(client)
try:
    zementis.delete_model(model_name)
except trendminer.ml.models.exceptions.MLModelNotFoundException:
    pass

# Now we can deploy the model and voila!
models = ZementisModels(client)
model_id = models.deploy_model(string)
model_details = models.model_details(model_id)